#  Práctica SDPD T3 – 2020

    Enrique Macip Belmonte
    Maria Ruiz Teixidor

## Instrucciones iniciales y setup

Antes de ejecutar este Notebook, se siguen los siguientes pasos:

1. Arranque de Zookeeper
2. Arranque de Kafka
3. Ejecutamos el archivo p_kafka_producer.py con la siguiente instrucción: `python p_kafka_producer.py 0.6 1.3 test data/occupancy_data.csv`. Con éste estamos indicando que se envíen los datos a la cola test de Kafka con un retardo variable entre muestras insertadas de entre 0.6 y 1.3 segundos.
4. Ahora ya se pueden ejecutar los jobs de Spark Streaming del Notebook.

## 1 Importaciones y creación del contexto

In [1]:
import os
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime
from operator import add
from operator import sub
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [2]:
# Load external packages programatically
import os
# THIS IS MANDATORY
# You must provide the information about the Maven artifact for the
# Spark Streaming connector to Kafka
# At present time, only the 0.8.2 version (deprecated) has
# Python support
#packages = "org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.5"
packages = "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5"
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)
# THIS IS COMPULSORY
# Comment the line below if JAVA_HOME is already set up or you
# only have a single JVM version in your system
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# OPTIONAL: Check setup of environment variables
print("PYSPARK_SUBMIT_ARGS = ",os.environ["PYSPARK_SUBMIT_ARGS"],"\n")
print("JAVA_HOME = ", os.environ["JAVA_HOME"])

PYSPARK_SUBMIT_ARGS =  --packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 pyspark-shell 

JAVA_HOME =  /Library/Java/JavaVirtualMachines/jdk1.8.0_31.jdk/Contents/Home


In [3]:
spark = SparkSession \
    .builder \
    .appName("prueba") \
    .getOrCreate()

In [4]:
spark

## 2 Fuente de datos - Lectura

Leemos los datos con la API de Spark Structured Streaming, indicando la cola de Kafka 'test':

In [5]:
df = spark \
    .readStream \
    .format("kafka")\
    .option("sep", ",") \
    .option("kafka.bootstrap.servers", 'localhost:9092')\
    .option('subscribe', 'test')\
    .load()

Observamos la estructura de los datos que llegan por Kafka:

In [6]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



Es necesario obtener los diferentes campos que contiene 'value', son los que nos interesan:

In [7]:
df = df.selectExpr('CAST(value AS STRING)')

df_data = df.select(
        split(df.value, ',')[0].alias("row").cast(StringType()),
        split(df.value, ',')[1].alias("date").cast(StringType()),
        split(df.value, ',')[2].alias("Temperature").cast(DoubleType()),
        split(df.value, ',')[3].alias("Humidity").cast(DoubleType()),
        split(df.value, ',')[4].alias("Light").cast(DoubleType()),
        split(df.value, ',')[5].alias("CO2").cast(DoubleType()),
        split(df.value, ',')[6].alias("HumidityRatio").cast(DoubleType()),
        split(df.value, ',')[7].alias("Occupancy").cast(StringType()))

Para poder realizar bien el ejercicio 2, convertimos la columna 'date' a un formato timestamp para que la ventana la detecte, ya que sólo se realizan por columnas de event time.

In [8]:
df_data = df_data.withColumn("date", regexp_replace(col("date"), '"', ''))
df_data = df_data.withColumn('date',to_timestamp("date", "yyyy-MM-dd HH:mm:ss"))

In [9]:
df_data = df_data.withColumn("row", regexp_replace(col("row"), '"', ''))
df_data = df_data.withColumn("row",df_data["row"].cast(IntegerType()))

# Ejercicio 1

**Calcular el promedio de valores de Temperatura, humedad relativa y concentración de CO2 para cada micro-batch y el promedio de dichos valores desde el arranque**

En primer lugar, realizamos la consulta con el modo de salida *update*, para cada micro-batch. Indicamos que los valores se vayan mostrando por la consola. Adicionalmente, especificamos que el intervalo de actualización sea de 5 segundos.

In [15]:
result_1_1 = (df_data.agg(avg(col("Temperature")).alias('MB-AVG Temperature'),
                          avg(col("Humidity")).alias('MB-AVG Humidity'),
                          avg(col("CO2")).alias('MB-AVG CO2'))
                         .writeStream
                         .format('console')
                         .trigger(processingTime= '5 seconds')
                         .outputMode("update")
                         .start())

In [16]:
result_1_1.stop()

En segundo lugar, realizamos la consulta con el modo de salida *complete*, para los valores desde el arranque:

In [13]:
# 1.2
result_1_2 = (df_data.agg(avg(col("Temperature")).alias('AVG Temperature'),
                        avg(col("Humidity")).alias('AVG Humidity'),
                        avg(col("CO2")).alias('AVG CO2'))
                        .writeStream
                        .format('console')
                        .trigger(processingTime= '5 seconds')
                        .outputMode("complete")
                        .start())

In [14]:
result_1_2.stop()

# Ejercicio 2

**Calcular el promedio de luminosidad en la estancia en ventanas deslizantes de tamaño 45 segundos, con un valor de deslizamiento de 15 segundos entre ventanas consecutivas.**

Especificamos las características de las ventanas en el groupBy, éstas se definen a partir de la columna date (formato timestamp). Para cada ventana se calcula el promedio de 'Light'. Indicamos que los valores se muestren por consola.

In [11]:
result_2 = (df_data.groupBy(window(col("date"), "45 seconds", "15 seconds"))
                   .agg(avg('Light').alias('Light_avg'))
                   .writeStream\
                   .format('console')\
                   .trigger(processingTime= '5 seconds')\
                   .outputMode("complete")\
                   .start())

In [12]:
result_2.stop()

# Ejercicio 3:

**Examinando los datos, podemos apreciar que el intervalo entre muestras originales no es exactamente de 1 minuto en muchos casos. Calcular el número de parejas de muestras consecutivas en cada micro-batch entre las cuales el intervalo de separación no es exactamente de 1 minuto.**

Opcion A con Strutured Streaming

In [10]:
df1_ww = (df_data.withColumn("row", col("row") + 1)
              .selectExpr("row AS row_pivot", "date AS timePivot") 
              .withWatermark("timePivot", "1 minutes")
          )

df2_ww = (df_data.selectExpr("row", "date AS time")       
                 .withWatermark("time", "1 minutes")
          )


# Join with event-time constraints
df = (df1_ww
      .join(df2_ww, expr(""" 
                         row_pivot = row AND 
                         time > timePivot AND
                         time < timePivot + interval 1 minutes
                         """))
                 .writeStream
                 .format('console')
                 .trigger(processingTime= '5 seconds')
                 .outputMode("append")
                 .start()
)

In [11]:
df.stop()

Intentamos hacer un cont del df resultante de join, de acuerdo con la documentación se podria hacer un groupBy (). Count () pero esta solución no nos devuelve

In [ ]:
Opcion B con Spark Streaming

In [ ]:
Es necesario apagar el kernel para ejecutarlo limpio desde el principio

In [2]:
import os
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime
from operator import add
from operator import sub
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

In [3]:
packages = "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.5"
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)
print("PYSPARK_SUBMIT_ARGS = ",os.environ["PYSPARK_SUBMIT_ARGS"],"\n")
print("JAVA_HOME = ", os.environ["JAVA_HOME"])

PYSPARK_SUBMIT_ARGS =  --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.5 pyspark-shell 

JAVA_HOME =  /Library/Java/JavaVirtualMachines/jdk1.8.0_31.jdk/Contents/Home


In [4]:
spark = (SparkSession
    .builder
    .appName("StructuredStreamingUCI")
    .getOrCreate())

spark

In [5]:
sc = SparkContext.getOrCreate()
ssc = StreamingContext(sc, 5)

In [6]:
kafkaParams = {"metadata.broker.list": "localhost:9092"}
stream = KafkaUtils.createDirectStream(ssc, ["test"], kafkaParams)
stream = stream.map(lambda o: str(o[1]))

In [7]:
def parseString(line):
      print(line)
      line = line.replace('"', '')
      s = line.split(",")
      try:
          return [{"row": int(s[0]),
                   "date": datetime.strptime(s[1], "%Y-%m-%d %H:%M:%S")}]
      except Exception as err:
          print("Wrong line format (%s): " % line)
          return []

In [8]:
orders = stream.flatMap(parseString)


In [9]:
#orders = orders.foreachRDD(rdd => { lag("date")})
orders.pprint()

In [10]:
ssc.start()

-------------------------------------------
Time: 2020-07-13 22:27:25
-------------------------------------------
{'row': 928, 'date': datetime.datetime(2015, 2, 5, 9, 17, 59)}
{'row': 929, 'date': datetime.datetime(2015, 2, 5, 9, 19)}
{'row': 930, 'date': datetime.datetime(2015, 2, 5, 9, 20)}
{'row': 931, 'date': datetime.datetime(2015, 2, 5, 9, 21)}
{'row': 932, 'date': datetime.datetime(2015, 2, 5, 9, 22)}
{'row': 933, 'date': datetime.datetime(2015, 2, 5, 9, 23)}
{'row': 934, 'date': datetime.datetime(2015, 2, 5, 9, 23, 59)}
{'row': 935, 'date': datetime.datetime(2015, 2, 5, 9, 24, 59)}
{'row': 936, 'date': datetime.datetime(2015, 2, 5, 9, 26)}
{'row': 937, 'date': datetime.datetime(2015, 2, 5, 9, 27)}
...

-------------------------------------------
Time: 2020-07-13 22:27:30
-------------------------------------------
{'row': 940, 'date': datetime.datetime(2015, 2, 5, 9, 29, 59)}
{'row': 941, 'date': datetime.datetime(2015, 2, 5, 9, 30, 59)}
{'row': 942, 'date': datetime.datetime(

In [11]:
ssc.stop(False)

In [51]:
# Como suele ser en pyspark
#df_data = df_data.withColumn("date_lag", lag("date")).over(window.partitionBy('date').orderBy("date"))

Tenemos un problema de no poder utilizar withColumn en un objeto DStreamer. Nos falto probar con RDD usando foreachRDD